In [1]:
# import packages
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

2024-04-30 10:32:23.027428: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 10:32:23.032439: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 10:32:23.102978: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 10:32:24.320355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# settings
save_directory    = '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS3/'
apply_grayscale   = True
reduce_dimensions = True
normalise_data    = True
resize_method     = tf.image.ResizeMethod.LANCZOS3
data_size_x       = 16
data_size_y       = data_size_x

In [3]:
# Save Summary
with open(save_directory + 'configurations.txt', 'w') as f:
    f.write(f'save_directory = {save_directory}\n')
    f.write(f'apply_grayscale = {apply_grayscale}\n')
    f.write(f'reduce_dimensions = {reduce_dimensions}\n')
    f.write(f'normalise_data = {normalise_data}\n')
    f.write(f'resize_method = {resize_method}\n')
    f.write(f'data_size_x = {data_size_x}\n')
    f.write(f'data_size_y = {data_size_y}\n')

In [4]:
# Dataset
cifar100 = tfds.builder('cifar100')

# Description of dataset
assert cifar100.info.features['image'].shape       == (32, 32, 3)
assert cifar100.info.features['label'].num_classes == 100
assert cifar100.info.splits['train'].num_examples  == 50000
assert cifar100.info.splits['test'].num_examples   == 10000

# Download and prepare the data
cifar100.download_and_prepare()
datasets = cifar100.as_dataset()


2024-04-30 10:32:35.716054: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

...

2024-04-30 10:32:51.059346: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Shuffling /home/ubuntu/tensorflow_datasets/cifar100/3.0.2.incompleteK0ZUCG/cifar100-train.tfrecord*...:   0%| …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /home/ubuntu/tensorflow_datasets/cifar100/3.0.2.incompleteK0ZUCG/cifar100-test.tfrecord*...:   0%|  …

Dataset cifar100 downloaded and prepared to /home/ubuntu/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


In [5]:
# Load data and convert to grayscale
if apply_grayscale:
    for split in ['train', 'test']:
        datasets[split] = datasets[split].map(lambda item: {
            'image': tf.image.rgb_to_grayscale(item['image']),
            'label': item['label']
        })

In [6]:
# Reduce dimensionality
if reduce_dimensions:
    for split in ['train', 'test']:
        datasets[split] = datasets[split].map(lambda item: {
            'image': tf.image.resize(
                item['image'],
                [data_size_x, data_size_y],
                method=resize_method, 
            ),
            'label': item['label']
        })

In [7]:
# Normalise
if normalise_data:
    # Take one example from the dataset
    for example in datasets['train'].take(1):
        image, label = example['image'], example['label']

    # Check maximal pixel value and normalise if above 1
    max_pixel_value = tf.reduce_max(image).numpy()
    if max_pixel_value > 1:    
        for split in ['train', 'test']:
            datasets[split] = datasets[split].map(lambda item: {
                'image': item['image'] / 255,
                'label': item['label']
            })


2024-04-30 10:33:55.546984: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-04-30 10:33:55.548619: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
# Save the datasets
for split in ['train', 'test']:
    datasets[split].save(save_directory + split)